## Prova 1 de C210 - L1

### **Instruções:**

- Prova individual e sem consulta, exceto ao material disponibilizado pelo laboratorio (https://github.com/alvaromfcunha-c210);
- Para responder as questões altere somente os trechos de código necessarios;
- Interpretação faz parte da prova;
- Ao finalizar a prova, enviar o arquivo de extensão `.ipynb` alterado com seu numero de matricula (ex.: `232.ipynb`).

### Boa prova!

### Questão 1 - Machine Learning:

Dada a situação hipotética: *a Marinha do Brasil está explorando um trecho do oceano usando um Sonar. Durante uma outra exploração foi coletado dados sobre o sinal do sonar e o objeto encontrado, que nesse caso era rochas e cilindros de metal. **É pedido para se criar um modelo de aprendizado de máquia capaz de utilizar os dados da última exploração capaz de categorizar o sinal do sonar para saber se o objeto é uma rocha (R) ou um cilindro de metal (M)**.*

Descomente **um** dos 3 trechos de código abaixo capaz de gerar esse modelo:

In [ ]:
import pandas as pd
dataset = pd.read_csv('sonar.csv')

dataset.head()

In [ ]:
import numpy as np

p = np.array(dataset.loc[:, dataset.columns != 'target'])
target = np.array(dataset['target'])

from sklearn.model_selection import train_test_split

p_train, p_test, target_train, target_test = train_test_split(p, target, test_size=0.2, random_state = 0)

# Classificacao
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(p_train, target_train)

# Regressao Linear
# from sklearn.linear_model import LinearRegression
# linear = LinearRegression()
# linear.fit(p_train, target_train)

# Agrupamento
# from sklearn.cluster import KMeans
# kmeans = KMeans(n_clusters=3)
# kmeans.fit(p_train)

### Questão 2 - PSO (Cornfield Vector)

Dado o código abaixo troque a função que mede distância da particula (distância euclidiana) para o objetivo para a função que mede o raio da circunferencia (distância da particula do marco zero). Dica: como o objetivo vai ser fixo no marco zero não vai ser necessário a variavel `posicao_alvo`.

Função raio da circunferencia:
```python
@staticmethod
def _cicle_radius(p_pos):
  return sum(np.array(p_pos)**2)
```

In [4]:
import numpy as np

class Particle:
  def __init__(self, dimensions, bounds, id):
    '''
    Construtor
    '''
    self.id = id
    self.dimensions = dimensions
    self.bounds = bounds
    self.position = []
    self.dist = np.inf
    self.velocity = []
    self.pbest_pos = []
    self.pbest_dist = np.inf

    # Definir valores aleatorios de velocidade e posicao.
    for i in range(self.dimensions):
      self.position.append(np.random.uniform(
        self.bounds[i][0], self.bounds[i][1]))
      self.velocity.append(np.random.uniform(
        self.bounds[i][0], self.bounds[i][1]))

  @staticmethod
  def _euclidean_distance(p1_pos, p2_pos):
    '''
    Calcula a distância euclidiana.
    '''
    pp1_pos = np.array(p1_pos)
    pp2_pos = np.array(p2_pos)
    distance = np.sqrt(sum((pp1_pos - pp2_pos)**2))

    return distance
  
  @staticmethod
  def _cicle_radius(p_pos):
    return sum(np.array(p_pos)**2)

  def evaluate(self, target_pos):
    '''
    Função que avalia e compara a proximidade da partículas em relação ao objetivo.
    '''
    self.dist = Particle._cicle_radius(self.position)

    if self.dist < self.pbest_dist:
      self.pbest_dist = self.dist
      self.pbest_pos = self.position

  def update_velocity(self, gbest_pos):
    '''
    Atualiza a velocidade com base no pbest e no gbest.
    '''
    for i in range(0, self.dimensions):
      r1 = np.random.uniform(0, 1)
      r2 = np.random.uniform(0, 1)

      vel_cognitive = r1 * (self.pbest_pos[i] - self.position[i])
      vel_social = r2 * (gbest_pos[i] - self.position[i])

      self.velocity[i] = (self.velocity[i] + vel_cognitive + vel_social) / 3

  def update_position(self):
    '''
    Atualiza a posição de cada uma das partículas.
    '''
    for i in range(0, self.dimensions):
      self.position[i] = self.position[i] + self.velocity[i]

      if self.position[i] < self.bounds[i][0]:
        self.position[i] = self.bounds[i][0]

      if self.position[i] > self.bounds[i][1]:
        self.position[i] = self.bounds[i][1]

class Swarm:

  def __init__(self, particles, target_pos):
    '''
    Construtor
    '''
    self.particles = particles
    self.target_pos = target_pos
    self.gbest_pos = []
    self.gbest_dist = np.inf

  def swarm_evaluate(self):
    '''
    Percorre todas as partículas e avalia/atualiza seu melhor pessoal e o melhor global do enxame.
    '''
    for p in self.particles:
      p.evaluate(self.target_pos)

      if p.dist < self.gbest_dist:
        self.gbest_pos = p.position
        self.gbest_dist = p.dist

  def swarm_update_velocities(self):
    '''
    Percorre todas as partículas e chama a função para atualizar a velocidade.
    '''
    for p in self.particles:
      p.update_velocity(self.gbest_pos)

  def swarm_update_positions(self):
    '''
    Percorre todas as partículas e chama a função para atualizar a posição.
    '''
    for p in self.particles:
      p.update_position()

from matplotlib import pyplot as plt
from PIL import Image
import glob
import os
import shutil

class PlotUtils:

  directory = "pso_plots"
  filename = 'pso.gif'

  @staticmethod
  def start_plot():
    if os.path.exists(PlotUtils.directory):
      shutil.rmtree(PlotUtils.directory)
    if not os.path.exists(PlotUtils.directory):
      os.makedirs(PlotUtils.directory)

  @staticmethod
  def plot_particle(particle):
    plt.scatter(particle.position[0], particle.position[1])

  @staticmethod
  def plot_iteration(i, bounds):
    plt.title(f"PSO {i}")
    plt.xlim(bounds[0][0], bounds[0][1])
    plt.ylim(bounds[1][0], bounds[1][1])
    plt.xlabel('x[0]')
    plt.ylabel('x[1]')
    iteration = str(i).zfill(5)
    plt.savefig(
      f"{PlotUtils.directory}/iteration_{iteration}.png", facecolor="white", dpi=75)
    plt.close()

  @staticmethod
  def save():
    images = [Image.open(f) for f in sorted(
      glob.glob(PlotUtils.directory+"/*"))]
    img = images[0]
    img.save(fp=PlotUtils.filename, format='GIF',
              append_images=images, save_all=True, duration=200, loop=0)
    if os.path.exists(PlotUtils.directory):
      shutil.rmtree(PlotUtils.directory)

def executar_pso(num_particulas, num_iteracoes,
                  posicao_alvo, num_dimencoes, limites):
  PlotUtils.start_plot()
  print("inicialização")

  particles = []
  for i in range(num_particulas):
    particles.append(Particle(num_dimencoes, limites, id=i))

  swarm = Swarm(particles, posicao_alvo)

  print("começando as iterações")
  i = 0
  while i < num_iteracoes:

    print(f"iteração {i}")
    # for p in swarm.particles:
    #   print(f'id{p.id} pbest: {p.pbest_dist}')
    # print(f'gbest: {swarm.gbest_dist}')

    swarm.swarm_evaluate()
    swarm.swarm_update_velocities()
    swarm.swarm_update_positions()

    for p in swarm.particles:
      PlotUtils.plot_particle(p)
    PlotUtils.plot_iteration(i, limites)

    i += 1

  PlotUtils.save()

executar_pso(
  num_particulas = 10,
  num_iteracoes = 20,
  posicao_alvo=[20, -20],
  num_dimencoes = 2,
  limites = [(-50, 50), (-50, 50)])

inicialização
começando as iterações
iteração 0
iteração 1
iteração 2
iteração 3
iteração 4
iteração 5
iteração 6
iteração 7
iteração 8
iteração 9
iteração 10
iteração 11
iteração 12
iteração 13
iteração 14
iteração 15
iteração 16
iteração 17
iteração 18
iteração 19


### Questão 3 - Algoritimo Genético (minimizar função)

**Item a)** troque a função a ser minimizada para `x²+y²+10`.

**Item b)** fixe o ponto de crossover para o terceiro ponto de intercessão dos genes.

**Item c)** na seleção dos pais em `selection` escolha os 2 melhores cromossomos usando a função `find_best_chromossome`. *Dica: para não selecionar os dois mesmos cromossomos retire o primeiro selecionado da cópia da população usando `population_copy = population.copy()` e `population_copy.remove(parent1)`*.


In [7]:
import numpy as np

class BitSet:
  def __init__(self, size):
    self.bits = np.full((1, size), False)

  def get(self, index):
    return self.bits[0, index]

  def set(self, index, value):
    self.bits[0, index] = value

  def flip(self, index):
    self.bits[0, index] = not self.bits[0, index]

  def debug(self):
    print(self.bits)

import random

class Chromossome:
  def __init__(self, x = None, y = None):
    '''
      Estancia o cromossomo com os valores de x e y (caso não sejam atribuidos são gerados valores aleatórios).
    '''
    if x == None:
      x = random.randint(-15, 15)

    if y == None:
      y = random.randint(-15, 15)

    self.__genes = Chromossome.get_genotype(x, y)

  def get_genes(self):
    return self.__genes

  def set_genes(self, genes):
    self.__genes = genes

  def __repr__(self):
    chr_str = "G = ["

    for i in range(10):
      chr_str += (i == 5 and " " or "") + (self.__genes.get(i) and "1" or "0")

    x, y = Chromossome.get_fenotype(self.__genes)

    chr_str += "], F = [" + str(x) + ", " + str(y) + "]"

    return chr_str

  @staticmethod
  def get_genotype(x, y):
    '''
      Método estático que retorna o genótipo dado o valor de x e y.
    '''
    bits = BitSet(10)

    xy_binary = "{:05b}".format(x) + "{:05b}".format(y)

    for i in range(10):
      bits.set(i, xy_binary[i] == '1')

    return bits

  @staticmethod
  def get_fenotype(genes):
    '''
      Método estático que retorna o valor de x e y dado o genótipo.
    '''
    x = (8 * genes.get(0) +
        4 * genes.get(1) +
        2 * genes.get(2) +
        1 * genes.get(3))

    if (genes.get(4) == 1):
        x *= -1

    y = (8 * genes.get(5) +
        4 * genes.get(6) +
        2 * genes.get(7) +
        1 * genes.get(8))

    if (genes.get(9) == 1):
        y *= -1

    return x, y

class Problem:
  @staticmethod
  def f(x, y):
    '''
      x²+y²
    '''
    return x**2 + y**2 + 10

  @staticmethod
  def f_chromossome(chromossome):
    '''
      Retorna f(x,y) dado o cromossomo.
    '''
    x, y = Chromossome.get_fenotype(chromossome.get_genes())
    return Problem.f(x, y)

  @staticmethod
  def g(x, y):
    '''
      1/(x²+y²) + 1
      Usado como função de fitness (função de avaliação).
      Valor varia de 0 à 1 e quanto mais próximo de 1, melhor.
    '''
    return 1 / (1 + Problem.f(x, y))

  @staticmethod
  def g_chromossome(chromossome):
    '''
      Retorna fitness dado o cromossomo.
    '''
    x, y = Chromossome.get_fenotype(chromossome.get_genes())
    return Problem.g(x, y)

  @staticmethod
  def f_average(population):
    '''
      Retorna a média de f(x,y) dos cromossomos dada a população.
    '''
    avg = 0
    for chromossome in population:
      avg += Problem.f_chromossome(chromossome)
    avg /= len(population)
    return avg

  @staticmethod
  def g_average(population):
    '''
      Retorna a média de fitness dos cromossomos dada a população.
    '''
    avg = 0
    for chromossome in population:
      x, y = chromossome.get_fenotype(chromossome.get_genes())
      avg += Problem.g(x, y)
    avg /= len(population)
    return avg

'''
**Item c)** na seleção dos pais em `selection` escolha os 2 melhores cromossomos usando a função `find_best_chromossome`. 
*Dica: para não selecionar os dois mesmos cromossomos retire o primeiro selecionado da cópia da população usando `population_copy = population.copy()` e `population_copy.remove(parent1)`*.
'''

class GeneticFunctions:
  @staticmethod
  def selection(population):
    '''
      Seleciona 2 cromossomos diferentes de forma aleatória.
    '''
    parent1 = GeneticFunctions.find_best_chromossome(population)
    population_copy = population.copy()
    population_copy.remove(parent1)
    parent2 = GeneticFunctions.find_best_chromossome(population)

    while parent1 is parent2:
      parent1 = random.choice(population)
      parent2 = random.choice(population)

    return parent1, parent2

  @staticmethod
  def crossover(population, parent1, parent2):
    '''
      Faz o cruzamento de 2 cromossomos, gerando 2 filhos e inserindo-os na população.
    '''
    crossover_point = 3

    parent1_genes = parent1.get_genes()
    parent2_genes = parent2.get_genes()

    child1_genes = BitSet(10)
    child2_genes = BitSet(10)

    for i in range(crossover_point):
      child1_genes.set(i, parent1_genes.get(i))
      child2_genes.set(i, parent2_genes.get(i))

    for i in range(crossover_point, 10):
      child1_genes.set(i, parent2_genes.get(i))
      child2_genes.set(i, parent1_genes.get(i))

    child1 = Chromossome()
    child1.set_genes(child1_genes)

    child2 = Chromossome()
    child2.set_genes(child2_genes)

    population.append(child1)
    population.append(child2)

    return crossover_point
      
  @staticmethod
  def mutation(population, mutation_prob):
    '''
      Gera mutação em 1 cromossomo aleatório da população dada a probabilidade de mutação.
    '''
    prob = random.uniform(0, 1)

    if prob < mutation_prob:
      target = random.choice(population)

      mutation_point = random.randint(0, 9)

      genes = target.get_genes()
      genes.flip(mutation_point)

      return True, mutation_point
    return False, None

  @staticmethod
  def elitism(population):
    '''
      Remove 2 dos piores cromossomos da população.
    '''
    for _ in range(2):
      worst_individual = GeneticFunctions.find_worst_chromossome(population)
      population.remove(worst_individual)
      
    return worst_individual

  @staticmethod
  def find_best_chromossome(population):
    '''
      Retorna o melhor cromossomo dada a população.
    '''
    best_chromossome = None

    for chromossome in population:
      score = Problem.g_chromossome(chromossome)

      if best_chromossome is None or score > Problem.g_chromossome(best_chromossome):
        best_chromossome = chromossome

    return best_chromossome

  @staticmethod
  def find_worst_chromossome(population):
    '''
      Retorna o pior cromossomo dada a população.
    '''
    worst_chromossome = None

    for chromossome in population:
      score = Problem.g_chromossome(chromossome)

      if worst_chromossome is None or score < Problem.g_chromossome(worst_chromossome):
        worst_chromossome = chromossome

    return worst_chromossome

def executar_ag(tam_populacao, n_geracoes, prob_mutacao):
  populacao = []
  for _ in range(tam_populacao):
    populacao.append(Chromossome())

  print('População inicial:', populacao)

  pontuacoes = []
  pontuacao_inicial = Problem.g_average(populacao)
  pontuacoes.append(pontuacao_inicial)

  print('Pontuação inicial:', pontuacao_inicial)

  medias_f = []
  media_f_inicial = Problem.f_average(populacao)
  medias_f.append(media_f_inicial)

  print('Média de f(x,y) inicial:', media_f_inicial)

  for geracao in range(n_geracoes):
    print('Geração:', geracao)

    pai1, pai2 = GeneticFunctions.selection(populacao)
    print('\tSelecionados:', pai1, pai2)

    pos_crossover = GeneticFunctions.crossover(populacao, pai1, pai2)
    print('\tCrossover entre selecionados na posição:', pos_crossover)

    teve_mutacao, pos_mutacao = GeneticFunctions.mutation(populacao, prob_mutacao)
    print('\tTeve mutação? ', teve_mutacao)
    if(teve_mutacao):
      print('\t\tNa posição:', pos_mutacao)
    
    pior_cromossomo = GeneticFunctions.elitism(populacao)
    print('Foi eliminado o pior cromossomo:', pior_cromossomo, Problem.g_chromossome(pior_cromossomo))

    pontuacao = Problem.g_average(populacao)
    print(f'Pontuação da geração {geracao}:', pontuacao)

    media_f = Problem.f_average(populacao)
    medias_f.append(media_f)
    print(f'Média de f(x,y) da geração {geracao}:', media_f)

    pontuacoes.append(pontuacao)

  melhor_cromossomo = GeneticFunctions.find_best_chromossome(populacao)
  
  return melhor_cromossomo, pontuacoes, medias_f

melhor_cromossomo, pontuacoes, medias_f = executar_ag(
  tam_populacao=10,
  n_geracoes=50,
  prob_mutacao=0.05
)

População inicial: [G = [01100 00011], F = [6, -1], G = [01000 01011], F = [4, -5], G = [01111 01110], F = [-7, 7], G = [01111 00101], F = [-7, -2], G = [00010 00000], F = [1, 0], G = [00100 00011], F = [2, -1], G = [00001 01111], F = [0, -7], G = [01000 01010], F = [4, 5], G = [00111 00010], F = [-3, 1], G = [01111 00111], F = [-7, -3]]
Pontuação inicial: 0.030870598496371332
Média de f(x,y) inicial: 49.3
Geração: 0
	Selecionados: G = [01111 00111], F = [-7, -3] G = [01100 00011], F = [6, -1]
	Crossover entre selecionados na posição: 3
	Teve mutação?  False
Foi eliminado o pior cromossomo: G = [01111 00111], F = [-7, -3] 0.014492753623188406
Pontuação da geração 0: 0.032036500637044114
Média de f(x,y) da geração 0: 43.2
Geração: 1
	Selecionados: G = [01000 01011], F = [4, -5] G = [01111 00111], F = [-7, -3]
	Crossover entre selecionados na posição: 3
	Teve mutação?  False
Foi eliminado o pior cromossomo: G = [01111 00111], F = [-7, -3] 0.014492753623188406
Pontuação da geração 1: 0.03